# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
env = gym.make('BipedalWalker-v2')

random_seed = 10
env.seed(random_seed)

# from agent import Agent
# agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], 
#               random_seed=random_seed)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[10]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [3]:
env.action_space, env.observation_space

(Box(4,), Box(24,))

In [4]:
action_size = env.action_space.shape[0]
state_size = env.observation_space.shape[0]

In [5]:
action_size, state_size

(4, 24)

In [6]:
env.observation_space.low, env.observation_space.high

(array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf], dtype=float32),
 array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       dtype=float32))

In [7]:
env.action_space.low, env.action_space.high

(array([-1., -1., -1., -1.], dtype=float32),
 array([1., 1., 1., 1.], dtype=float32))

### 3. Instantiate DDPG Agent

In [8]:
from agent2 import Agent

agent = Agent(s_size=state_size, a_size=action_size, random_seed=random_seed)

### 4. Train the Agent with DDPG

In [9]:
from collections import deque
import numpy as np
import time
import torch

n_episodes=1000
# n_steps=1000
n_learn=1
# learn_every=20
R_goal=0 # TOCHECK
Rs_deque = deque(maxlen=100)
meanRs = [] # plot
Rs = [] # plot
dlosses = []
glosses = []

In [ ]:
for i_episode in range(n_episodes):
    
    # # Reset Env
    # env_info = env.reset(train_mode=train_mode)[brain_name]       # reset the environment    
    # S = env_info.vector_observations                   # get the current state (for each agent)
    s = env.reset() # s: state [-1:1, -1:1, -8:8]
    #print(s.shape)

    #Rs = np.zeros(num_agents)                            # initialize the score (for each agent)
    R = 0 # R: total reward
    
    glosses_batch = []
    dlosses_batch = []

    #for i_step in range(n_steps):
    while True:
        s = np.reshape(s, [1, -1])
        a = agent.act(s) # a=[-2, 2]
        #a *= 2
        #print(a.shape)
        
        # Exploration noise
        # noise = np.random.normal(loc=0, scale=0.1, size=[num_agents, action_size])
        # A += noise
        # A = np.clip(a=A, a_min=-1, a_max=1)

        # env_info = env.step(A)[brain_name]            # send all actions to the environment
        # S2 = env_info.vector_observations          # get next state (for each agent)
        # rewards = env_info.rewards                          # get reward (for each agent)
        # dones = env_info.local_done                         # see if episode finished
        #s2: next_state, r: reward, done/terminal, env: environment
        a = np.reshape(a, [-1])
        s2, r, done, _ = env.step(a)
        #print(s2.shape, r, done)

        #s2_, q_ = agent.env(s, a)
        a = np.reshape(a, [1, -1])
        #s2_, _ = agent.env(s, a)
        # print(s2_)
        # s2_ = np.tanh(s2_) # predicted next state [-1:1, -1:1, -8:8]
        # print(s2_)
        # s2_[2] = s2_[2] * 8
        # print(s2_)

        # for s, a, reward, s2, done in zip(S, A, rewards, S2, dones):
        # agent.step(state, action, reward, next_state, done) # send actions to the agent
        agent.step(s, a, r, s2, done)

        R += r                           # update the score (for each agent)
        s = s2                                # roll over the states to the next state

        # if i_step%learn_every == 0:
        dlosses_Nlearn = []
        for _ in range(n_learn):
            dloss = agent.start_dlearn()
            #print(dloss)
            dlosses_Nlearn.append(dloss)
        dlosses_batch.append(np.mean(dlosses_Nlearn))
            
        #print(len(agent.memory))
        gloss = agent.start_glearn()
        glosses_batch.append(gloss)

        # if np.any(dones):                                   # exit loop if episode finished
        #     break
        if done:
            break

    #meanR = np.mean(Rs)
    #Rs_deque.append(meanR)
    Rs_deque.append(R)
    meanRs.append(np.mean(Rs_deque)) # plot
    Rs.append(R) # plot
    dlosses.append(np.mean(dlosses_batch))
    glosses.append(np.mean(glosses_batch))

    print('\rEpisode {}\tTotal Average Score: {:.2f}\tGloss: {:.2f}, \tDloss: {:.2f}'.format(
           i_episode+1, np.mean(Rs_deque), np.mean(glosses_batch), np.mean(dlosses_batch)))
    
    if np.mean(Rs_deque) >= R_goal:
        torch.save(agent.g.state_dict(), 'g-bipedal.pth')
        torch.save(agent.d.state_dict(), 'd-bipedal.pth')
        break

Episode 1	Total Average Score: -92.41	Gloss: 0.00, 	Dloss: 0.00
Episode 2	Total Average Score: -92.44	Gloss: 0.00, 	Dloss: 0.00
Episode 3	Total Average Score: -92.44	Gloss: 0.00, 	Dloss: 0.00
Episode 4	Total Average Score: -92.46	Gloss: 0.00, 	Dloss: 0.00
Episode 5	Total Average Score: -92.47	Gloss: 0.00, 	Dloss: 0.00
Episode 6	Total Average Score: -92.49	Gloss: 0.00, 	Dloss: 0.00
Episode 7	Total Average Score: -92.49	Gloss: 0.00, 	Dloss: 0.00
Episode 8	Total Average Score: -92.48	Gloss: 0.00, 	Dloss: 0.00
Episode 9	Total Average Score: -92.48	Gloss: 0.00, 	Dloss: 0.00
Episode 10	Total Average Score: -92.47	Gloss: -0.01, 	Dloss: 64.64
Episode 11	Total Average Score: -86.35	Gloss: 0.02, 	Dloss: 55.00
Episode 12	Total Average Score: -88.92	Gloss: 0.05, 	Dloss: 30.80
Episode 13	Total Average Score: -91.59	Gloss: 0.05, 	Dloss: 31.22
Episode 14	Total Average Score: -92.60	Gloss: 0.06, 	Dloss: 29.69
Episode 15	Total Average Score: -88.71	Gloss: 0.08, 	Dloss: 21.26
Episode 16	Total Average Sc

Episode 126	Total Average Score: -110.84	Gloss: 1.83, 	Dloss: 9.89
Episode 127	Total Average Score: -111.06	Gloss: 1.84, 	Dloss: 9.10
Episode 128	Total Average Score: -111.25	Gloss: 1.85, 	Dloss: 9.03
Episode 129	Total Average Score: -111.48	Gloss: 1.86, 	Dloss: 9.27
Episode 130	Total Average Score: -112.02	Gloss: 1.91, 	Dloss: 9.48
Episode 131	Total Average Score: -112.26	Gloss: 1.94, 	Dloss: 10.87
Episode 132	Total Average Score: -112.51	Gloss: 1.96, 	Dloss: 9.07
Episode 133	Total Average Score: -112.73	Gloss: 1.96, 	Dloss: 8.65
Episode 134	Total Average Score: -112.97	Gloss: 1.97, 	Dloss: 9.98
Episode 135	Total Average Score: -113.17	Gloss: 1.98, 	Dloss: 8.90
Episode 136	Total Average Score: -113.27	Gloss: 1.99, 	Dloss: 10.28
Episode 137	Total Average Score: -113.31	Gloss: 1.99, 	Dloss: 9.09
Episode 138	Total Average Score: -113.32	Gloss: 2.00, 	Dloss: 9.18
Episode 139	Total Average Score: -113.33	Gloss: 2.00, 	Dloss: 9.23
Episode 140	Total Average Score: -113.33	Gloss: 2.00, 	Dloss

Episode 248	Total Average Score: -110.59	Gloss: 3.33, 	Dloss: 5.84
Episode 249	Total Average Score: -110.72	Gloss: 3.39, 	Dloss: 5.73
Episode 250	Total Average Score: -110.58	Gloss: 3.45, 	Dloss: 5.46
Episode 251	Total Average Score: -110.84	Gloss: 3.55, 	Dloss: 4.96
Episode 252	Total Average Score: -110.84	Gloss: 3.64, 	Dloss: 4.44
Episode 253	Total Average Score: -110.79	Gloss: 3.65, 	Dloss: 4.49
Episode 254	Total Average Score: -110.89	Gloss: 3.65, 	Dloss: 3.99
Episode 255	Total Average Score: -111.02	Gloss: 3.66, 	Dloss: 4.32
Episode 256	Total Average Score: -111.10	Gloss: 3.66, 	Dloss: 4.32
Episode 257	Total Average Score: -111.21	Gloss: 3.67, 	Dloss: 4.82
Episode 258	Total Average Score: -111.22	Gloss: 3.67, 	Dloss: 5.66
Episode 259	Total Average Score: -111.28	Gloss: 3.68, 	Dloss: 4.83
Episode 260	Total Average Score: -111.27	Gloss: 3.69, 	Dloss: 4.53
Episode 261	Total Average Score: -111.32	Gloss: 3.70, 	Dloss: 4.72
Episode 262	Total Average Score: -111.43	Gloss: 3.71, 	Dloss: 

Episode 371	Total Average Score: -110.54	Gloss: 5.39, 	Dloss: 3.05
Episode 372	Total Average Score: -110.64	Gloss: 5.40, 	Dloss: 3.09
Episode 373	Total Average Score: -110.77	Gloss: 5.52, 	Dloss: 2.84
Episode 374	Total Average Score: -110.97	Gloss: 5.65, 	Dloss: 2.86
Episode 375	Total Average Score: -111.38	Gloss: 5.68, 	Dloss: 2.81
Episode 376	Total Average Score: -111.48	Gloss: 5.70, 	Dloss: 2.78
Episode 377	Total Average Score: -111.46	Gloss: 5.72, 	Dloss: 3.25
Episode 378	Total Average Score: -111.44	Gloss: 5.74, 	Dloss: 3.00
Episode 379	Total Average Score: -111.42	Gloss: 5.76, 	Dloss: 2.84
Episode 380	Total Average Score: -111.15	Gloss: 5.77, 	Dloss: 2.87
Episode 381	Total Average Score: -111.09	Gloss: 5.79, 	Dloss: 2.88
Episode 382	Total Average Score: -110.98	Gloss: 5.81, 	Dloss: 2.98
Episode 383	Total Average Score: -111.07	Gloss: 5.83, 	Dloss: 2.84
Episode 384	Total Average Score: -111.16	Gloss: 5.84, 	Dloss: 2.80
Episode 385	Total Average Score: -111.22	Gloss: 5.85, 	Dloss: 

Episode 494	Total Average Score: -113.48	Gloss: 7.05, 	Dloss: 2.26
Episode 495	Total Average Score: -113.55	Gloss: 7.07, 	Dloss: 2.07
Episode 496	Total Average Score: -113.65	Gloss: 7.09, 	Dloss: 2.11
Episode 497	Total Average Score: -113.51	Gloss: 7.11, 	Dloss: 2.16
Episode 498	Total Average Score: -113.44	Gloss: 7.12, 	Dloss: 2.17
Episode 499	Total Average Score: -113.34	Gloss: 7.14, 	Dloss: 2.10
Episode 500	Total Average Score: -113.24	Gloss: 7.15, 	Dloss: 2.14
Episode 501	Total Average Score: -113.21	Gloss: 7.17, 	Dloss: 2.08
Episode 502	Total Average Score: -113.10	Gloss: 7.18, 	Dloss: 2.01
Episode 503	Total Average Score: -113.08	Gloss: 7.19, 	Dloss: 2.07
Episode 504	Total Average Score: -113.14	Gloss: 7.20, 	Dloss: 2.10
Episode 505	Total Average Score: -113.16	Gloss: 7.21, 	Dloss: 2.02
Episode 506	Total Average Score: -113.27	Gloss: 7.23, 	Dloss: 2.08
Episode 507	Total Average Score: -113.03	Gloss: 7.24, 	Dloss: 1.97
Episode 508	Total Average Score: -112.87	Gloss: 7.25, 	Dloss: 

### 5. Plot the result

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

plt.plot(np.arange(1, len(meanRs)+1), meanRs)
plt.ylabel('total_average_scores')
plt.xlabel('Episode #')
plt.show()

### 6. Watch the agent running with saved weights

In [36]:
# Load the saved weights into Pytorch model
agent.g.load_state_dict(torch.load('g-bipedal.pth', map_location='cpu'))
agent.d.load_state_dict(torch.load('d-bipedal.pth', map_location='cpu'))

# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# S = env_info.vector_observations                  # get the current states (S) (for each agent)
s = env.reset() # s: state # get the current state (s) (for an agent)
s_ = np.zeros_like(s)# s_: predicted state # predict the current state (s) (by the agent)
# print(s.shape)

#Rs = np.zeros(num_agents)                          # initialize the total scores (Rs) (for each agent)
R = 0 # R:total reward # initialize the total score (R) (for an agent)

while True:
    #A = agent.act(S)                        # select actions (A) from loaded model agents
    s = np.reshape(s, [1, -1])
    s_ = np.reshape(s_, [1, -1])
    a = agent.act(s, s_) # a: [-1, +1]    # select action (a) from loaded model agent
    #a = agent.act(s)
    #print(s.shape, a.shape)
    
    # env_info = env.step(A)[brain_name]           # send all actions (A) to tne environment (env)
    # S2 = env_info.vector_observations         # get next states (S2) (for each agent)
    # rewards = env_info.rewards                         # get rewards (for each agent)
    # dones = env_info.local_done                        # see if the episode is done/finished (terminal)
    s2, r, done, _ = env.step(np.reshape(a, [-1]))
    #s2, r, done = env.step(a)
    
    #s2_, q_ = agent.env(s, a)
    s2_, _ = agent.env(s, a)
    
    # Rs += env_info.rewards                         # update the total scores (Rs) (for each agent)
    # S = S2                               # roll over current states (S) to next states (S2)
    R += r # update the total score (R) (for an agent)
    s = s2 # roll over current state (s) to next state (s2)
    s_ = s2_ # roll over the predcited current state (s) to the predicted next state (s2_)
    
    #if np.any(dones):                                  # exit loop if episode is done/finished
    if done: # exit loop if episode is done/finished (terminal)
        break
        
print('Average of total scores: {}'.format(R))

Average of total scores: -106.75700939138544


When finished, you can close the environment.

In [37]:
env.close()